In [1]:
import pandas as pd
import numpy as np
import sidetable
import requests
from time import sleep
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import warnings
warnings.filterwarnings('ignore')

# CSV Inicial

Nos descargamos el CSV que vamos a utilizar de Kaggle.

Los datos son sobre casas en los Estados de Washington, California y Oregón, y divididos por ciudades. Incluye muchas de las ciudades más importantes de cada estado. Los datos incluyen el precio, dirección, número de cuartos, baños, el área de la casa, etc.

Link al kaggle: https://www.kaggle.com/ialjadani/houses-prices-in-selected-area-in-usa

In [2]:
df = pd.read_csv ("../Data/01. CSV_inicial_houses_USA.csv")
df.sort_values(by = ["City"]).head(3)

,Unnamed: 0,Price,Currency,Brokered_by,AD_Situation,Rooms,Bath,Building space,Measruing unit,Address,City,State
1182,1182,899000,USD,Redfin Corporation,New - 18 hours ago,4,2.5,1860,sqft,"301 S Cordova St, Alhambra, CA 91801",Alhambra,CA 91801
1234,1234,510000,USD,Keller Williams Signature,New - 20 hours ago,2,2.0,1123,sqft,"777 E Valley Blvd Apt 56, Alhambra, CA 91801",Alhambra,CA 91801
1302,1302,1188000,USD,COMPASS,New,3,2.5,1890,sqft,"424 N Cordova St, Alhambra, CA 91801",Alhambra,CA 91801


In [3]:
df.shape

(1718, 12)

# Selenium

A continuación, mediante Selenium, vamos a extraer información adicional sobre índices (de poder adquisitivo, calidad, seguridad, sanidad, etc.) de las 11 ciudades más importantes y relevantes de estos 3 Estados. Esto lo usaremos más adelante para enriquecer nuestro CSV inicial

Lo primero que vamos a hacer es definir una serie de opciones para trabajar con Selenium

In [4]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [5]:
# iniciamos el driver
driver = webdriver.Chrome(ChromeDriverManager().install())
## accedemos a la pagina web

driver.get("https://es.numbeo.com/calidad-de-vida/pa%25C3%25ADs/Estados-Unidos")
driver.implicitly_wait(15)
# esperamos

driver.find_element_by_css_selector("#content_and_logo > table:nth-child(10) > tbody").text



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/monicagoizuetabeltran/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


'Índice de Poder Adquisitivo 104,42   Alto\nÍndice de Seguridad 51,72   Moderado\nÍndice de Sanidad 69,06   Alto\nÍndice de Clima 77,33   Alto\nÍndice de Costo de Vida 70,11   Moderado\nRelación Precio/Ingresos para Propiedades 3,93   Muy Bajo\nÍndice de Tiempo de Desplazamiento en Tráfico 32,88   Bajo\nÍndice de Contaminación 35,44   Bajo\nƒ Índice de Calidad de Vida: 170,01   Muy alto'

Hemos accedido a la información genérica de Estados Unidos, perfecto! Y ahora vamos a acceder a la información de las ciudades que nos interesan:

In [6]:
ciudades_list = ["Olympia","Portland","Seattle","Anaheim","Bellevue","Irvine","Playa-Larga","Los-Ángeles","Pasadena","Portland","Redmond","Tacoma","Vancouver"]         

In [7]:
url_list = []

for i in ciudades_list:
    url_list.append(f"https://es.numbeo.com/calidad-de-vida/ciudad/{i}")

url_list

['https://es.numbeo.com/calidad-de-vida/ciudad/Olympia',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Portland',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Seattle',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Anaheim',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Bellevue',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Irvine',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Playa-Larga',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Los-Ángeles',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Pasadena',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Portland',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Redmond',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Tacoma',
 'https://es.numbeo.com/calidad-de-vida/ciudad/Vancouver']

Ya tenemos la lista de las url que queremos! Ahora vamos a sacar la tabla para cada una de las url de arriba:

In [8]:
resultados = []
for z in url_list:
    driver.get(z)
    driver.implicitly_wait(15)
    res = driver.find_element_by_xpath('/html/body/div[2]/table[2]/tbody').text
    resultados.append(res)

WebDriverException: Message: unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=99.0.4844.74)
Stacktrace:
0   chromedriver                        0x000000010c7eb159 chromedriver + 5120345
1   chromedriver                        0x000000010c778b13 chromedriver + 4651795
2   chromedriver                        0x000000010c368d1f chromedriver + 392479
3   chromedriver                        0x000000010c353b53 chromedriver + 306003
4   chromedriver                        0x000000010c352bf8 chromedriver + 302072
5   chromedriver                        0x000000010c35313c chromedriver + 303420
6   chromedriver                        0x000000010c353094 chromedriver + 303252
7   chromedriver                        0x000000010c359db9 chromedriver + 331193
8   chromedriver                        0x000000010c35a082 chromedriver + 331906
9   chromedriver                        0x000000010c354448 chromedriver + 308296
10  chromedriver                   

In [ ]:
resultados

A continuación vamos a crear un diccionario, pasándole los Key-values, y con objetivo de crear un DataFrame:

In [ ]:
key_list = ["Olympia","Portland","Seattle","Anaheim","Bellevue","Irvine","Long Beach","Los Ángeles","Pasadena","Portland","Redmond","Tacoma","Vancouver"]
value_list = resultados

In [ ]:
dict_from_list = dict(zip(key_list, value_list))
#print(dict_from_list)

In [ ]:
df1 = pd.DataFrame(dict_from_list, index = ["indice1"]).T
df1

A continuación vamos a split la columna "indice1" por los espacios! Y resetemos el índice:

In [ ]:
df2 = (df1["indice1"].str.split("\n", expand=True)).reset_index()
df2.head(1)

Actualizamos el nombre de las columnas

In [ ]:
df2.columns = ['City', 'índice_poder_adq', 'índice_seguridad','índice_sanidad', 'índice_clima', 'índice_costo_vida', 'relación_precio_ingresos_propiedades', 'índice_costo_desplazamiento', 'índice_contaminación', 'índice_calidad_vida']
df2.head(1)

Creamos nuevas columnas, quedándonos únicamente con los valores numéricos:

In [ ]:
df2["Índice_poder_adqui"] = df2.índice_poder_adq.str.extract("(\d+,\d+)", expand=True)
df2["Índice_seguridad"] = df2.índice_seguridad.str.extract("(\d+,\d+)", expand=True)
df2["Índice_sanidad"] = df2.índice_sanidad.str.extract("(\d+,\d+)", expand=True)
df2["Índice_clima"] = df2.índice_clima.str.extract("(\d+,\d+)", expand=True)
df2["Índice_costo_vida"] = df2.índice_costo_vida.str.extract("(\d+,\d+)", expand=True)
df2["Relación_precio_ingresos_propiedades"] = df2.relación_precio_ingresos_propiedades.str.extract("(\d+,\d+)", expand=True)
df2["Índice_costo_desplazamiento"] = df2.índice_costo_desplazamiento.str.extract("(\d+,\d+)", expand=True)
df2["Índice_contaminación"] = df2.índice_contaminación.str.extract("(\d+,\d+)", expand=True)
df2["Índice_calidad_vida"] = df2.índice_calidad_vida.str.extract("(\d+,\d+)", expand=True)
df2.head(1)

Eliminamos las columnas que ya no necesitamos porque la información está duplicada:

In [ ]:
eliminar = ['índice_poder_adq', 'índice_seguridad', 'índice_sanidad',
       'índice_clima', 'índice_costo_vida',
       'relación_precio_ingresos_propiedades', 'índice_costo_desplazamiento',
       'índice_contaminación', 'índice_calidad_vida']

In [ ]:
df2_limpio = df2.drop(eliminar, axis = 1)
df2_limpio.head(1)

Exportamos el DataFrame a un CSV

In [ ]:
df2_limpio.to_csv("01. Resultado_Selenium_índices_ciudades_EEUU.csv")